<a href="https://colab.research.google.com/github/Surajit96-lang/codex_techno_3/blob/main/CODEX_TECHNO_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving AB_NYC_2019.csv to AB_NYC_2019.csv


In [ ]:
data=pd.read_csv('AB_NYC_2019.csv', encoding='latin1')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [ ]:
print(data.columns)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


In [ ]:
data.isnull().sum()

,0
id,0
name,16
host_id,0
host_name,21
neighbourhood_group,0
neighbourhood,0
latitude,0
longitude,0
room_type,0
price,0


In [ ]:
data['reviews_per_month'] = data['reviews_per_month'].fillna(0)
data['name'] = data['name'].fillna(0)
data['host_name'] = data['host_name'].fillna(0)
data['last_review'] = data['last_review'].fillna(0)
data.isnull().sum()

,0
id,0
name,0
host_id,0
host_name,0
neighbourhood_group,0
neighbourhood,0
latitude,0
longitude,0
room_type,0
price,0


In [ ]:
data.duplicated().sum()
df_clean = data.drop_duplicates(subset=['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'])
print(df_clean.shape)

(48895, 16)


In [ ]:
data['price'] = data['price'].astype(float)
data['last_review'] = pd.to_datetime(data['last_review'], format='%d-%m-%y',errors='coerce')
print(data.dtypes[['price','last_review']])

price                 float64
last_review    datetime64[ns]
dtype: object


In [ ]:
data.columns = (data.columns.str.strip()
              .str.lower())
print(data.columns)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


In [ ]:
date_cols = [col for col in data.columns if "date" in col.lower()]
for col in date_cols:
    data[col] = pd.to_datetime(data[col], errors="coerce")
print(data.dtypes[date_cols])

Series([], dtype: object)


OUTLIER DETECTION

In [ ]:
# 1. Select only numeric columns
num_cols = data.select_dtypes(include=[np.number]).columns
# 2. Function to detect outliers using IQR
def detect_outliers_iqr(data, col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = data[(data[col] < lower) | (data[col] > upper)]
    return outliers, lower, upper
# 3. Apply detection on all numeric columns
outlier_summary = {}

for col in num_cols:
    outliers, lower, upper = detect_outliers_iqr(data, col)
    outlier_summary[col] = {
        "outlier_count": len(outliers),
        "lower_bound": lower,
        "upper_bound": upper
    }

# Convert summary to DataFrame for easy viewing
outlier_report = pd.DataFrame(outlier_summary).T

# 4. Example handling strategies
data_no_outliers = data.copy()
for col in num_cols:
    _, lower, upper = detect_outliers_iqr(data_no_outliers, col)
    data_no_outliers = data_no_outliers[(data_no_outliers[col] >= lower) & (data_no_outliers[col] <= upper)]

# Option B: Cap/Floor outliers (Winsorization)
data_capped = data.copy()
for col in num_cols:
    _, lower, upper = detect_outliers_iqr(data_capped, col)
    data_capped[col] = np.where(data_capped[col] < lower, lower,
                       np.where(data_capped[col] > upper, upper, data_capped[col]))

# Save results
outlier_report.to_csv("AB_NYC_2019_outlier_report.csv")
data_no_outliers.to_csv("AB_NYC_2019_no_outliers.csv", index=False)
data_capped.to_csv("AB_NYC_2019_capped.csv", index=False)

print("Outlier detection completed. Reports and cleaned datasets saved.")

Outlier detection completed. Reports and cleaned datasets saved.
